# Convert ATT&CK 6.x techniques to 7.x techniques
### Input: file with list of v6 MITRE techniques in Txxxx format
### Check if any of the techniques had been replaced by 7.x techniques
### Output a list of attack pattern ids for use by further analysis of MITRE in STIX format



### Import required Python libraries and define input and output file paths

In [ ]:
from attackcti import attack_client
import json

inputFile = 'v6file' #enter the full path and filename of text file containing version 6 ATT&CK technique numbers, line separated, including the T (eg. "T1033")
outputFile = 'v7file' #enter the full path and filename to which the version 7 techniques should be written. File does not need to exist.


### Load the MITRE ATT&CK framework into the all_enterprise variable

In [ ]:
lift = attack_client()
%time all_enterprise = lift.get_all_enterprise()

### Load list of techniques 

In [ ]:
techniques = []
techniques = [line.rstrip() for line in open(inputFile)]


### List techniques 

In [ ]:
print(techniques)

### Convert the list of technique IDs to attack pattern IDs. For each technique in the list, check if it has been revoked and replaced by a newer techniqueID. If so, look up the new attack pattern ID.

In [ ]:
# note that the source_ref is replaced by the target_ref
attackIDs = []
for technique in techniques:
    for t in all_enterprise['techniques']:
        if t['external_references'][0]['external_id'] == technique:
            if t['revoked'] == True: #note that this key doesn't exist if the technique wasn't replaced
                for r in all_enterprise['relationships']:
                    if r['relationship_type'] == 'revoked-by' and r['source_ref'] == t['id']:
                        attackIDs.append(r['target_ref'])
            else:
                attackIDs.append(t['id'])
    


### Output a list of the new techniqueID numbers (for manual verification)

In [ ]:
newTechniques = []
for i in attackIDs:
    for t in all_enterprise['techniques']:
        if t['id'] == i:
            newTechniques.append(t['external_references'][0]['external_id'])

In [ ]:
print(newTechniques)

### Write out the v7 techniques to a file

In [ ]:
f = open(outputFile, 'a')
for t in newTechniques:
    f.write(t+'\n')
f.close()
